In [51]:
from selenium.webdriver.common.by import By
from time import sleep
import re
import glob
import os
import json
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
%load_ext autoreload
%load_ext dotenv
%autoreload
%dotenv ../.env

chrome_options = Options()
chrome_options.add_experimental_option("debuggerAddress", "127.0.0.1:9014")
driver = webdriver.Chrome(options=chrome_options)
DOWNLOAD_FOLDER = '../downloads'
title = current = button = paginator = page_els = ''
nome_atual = ''
JSON_FILE = 'linkas.json'

def load_items():
    global title, current, button, paginator, page_els
    title = driver.find_element(By.CSS_SELECTOR, '#elementoDepura h2')
    current = driver.find_element(By.CSS_SELECTOR, '#elementoDepura span')
    button = driver.find_element(By.CSS_SELECTOR, '#elementoDepura button')
    paginator = driver.find_element(By.CSS_SELECTOR, '.artdeco-pagination__pages.artdeco-pagination__pages--number')
    page_els = paginator.find_elements(By.CSS_SELECTOR, 'li')

def create_file_if_not_exists():
    try:
        f = open(JSON_FILE, 'r')
        if not f.read().strip():
            f.close()
            f = open(JSON_FILE, 'w')
            f.write('{}')
    except FileNotFoundError:
        f = open(JSON_FILE, 'w')
        f.write('{}')
    finally:
        f.close()

def get_current_name():
    create_file_if_not_exists()
    try:
        f = open(JSON_FILE, 'r')
        _dict = json.load(f)
        current_index = 0 if not _dict else str(max([int(x) for x in _dict.keys()]))
        current_name = '' if current_index == 0 else _dict[current_index]['nome_completo']
    except FileNotFoundError:
        f = open(JSON_FILE, 'w')
        f.write('{}')
        current_name = ''
    finally:
        f.close()
    return current_name

def start():
    current_name = get_current_name()
    driver.get(os.getenv('URL_LINKEDIN_HUNGARY'))
    sleep(1)
    driver.execute_script("let el = document.createElement('div'); el.style='border: 2px #000; background: #fff; padding: 30px; position: fixed; top: 0; left: 0; z-index: 99999'; document.querySelector('body').appendChild(el); let title = document.createElement('h2'); let current = document.createElement('span'); el.appendChild(title); el.appendChild(current); title.innerText = 'Nome atual'; title.style = 'font-size: 20px; color: #111'; let stop = document.createElement('button'); el.appendChild(stop); current.style = 'font-size: 20px; font-weight: 300; text-transform: uppercase'; stop.innerText = 'Pausar'; stop.style = 'color: #f00; font-size: 20px; text-transform: uppercase'; stop.onclick = (e) => {console.log(e); if(e.target.innerText == 'PAUSAR') {e.target.innerText = 'RETOMAR'; e.target.style.color = '#0f0';} else {e.target.innerText = 'PAUSAR'; e.target.style.color = '#f00'}}; current.innerText = '" + ('FULANO' or current_name) + "'; el.style.display = 'flex'; el.style.flexDirection = 'column'; el.style.alignItems = 'center'; el.style.justifyContent = 'center'; el.id = 'elementoDepura'")
    # driver.execute_script("document.body.style.zoom = '80%'")
    load_items()
    sleep(1)

def sel(selector, el=None):
    if el:
        el.find_element(By.CSS_SELECTOR, selector)
    return driver.find_element(By.CSS_SELECTOR, selector)

def sels(selector, el=None):
    if el:
        el.find_elements(By.CSS_SELECTOR, selector)
    return driver.find_elements(By.CSS_SELECTOR, selector)

def clicar(el):
    webdriver.ActionChains(driver).move_to_element(el).click(el).perform()

def change_current_name(name):
    driver.execute_script(f"document.querySelector('#elementoDepura span').innerText = '{name}'")

def put_on_classify_by_date():
    classify_by_el = sel('.artdeco-pill.artdeco-pill--slate.artdeco-pill--3.artdeco-pill--toggle.artdeco-pill--selected.artdeco-dropdown__trigger.artdeco-dropdown__trigger--placement-bottom.ember-view')
    classify_by_el.click()
    sleep(1)
    parent_el = 'div.artdeco-dropdown__content-inner'
    data_relevancia = sel(f'{parent_el} ul > li:nth-child(2)')
    data_relevancia.click()
    sleep(1)
    confirmar = sel(f'{parent_el} button.artdeco-button.artdeco-button--primary')
    confirmar.click()
    sleep(1)
    sel('body').click()

def get_json_dict():
    with open(JSON_FILE, 'r', encoding='utf-8') as _json:
        reading = ''.join(_json.read()).strip().replace('\n', '')
        reading = reading or '{}'
        return json.loads(reading)

def get_current_json_index(_dict=None):
    create_file_if_not_exists()
    _dict = _dict or get_json_dict()
    return max([int(x) for x in _dict.keys()] or [0])

def update_json(data: dict):
    create_file_if_not_exists()
    _dict = get_json_dict()
    with open(JSON_FILE, 'w', encoding='utf-8') as _json:
        current = get_current_json_index(_dict) + 1
        _dict[current] = data
        _json.write(json.dumps(_dict, ensure_ascii=False))

def invert(index, _max):
    inverted_index = -(_max - index)
    return inverted_index if inverted_index < 0 else 0

def get_normal_index(index, _max):
    normal_index = index if index >= 0 else index + _max
    return normal_index

def open_page(page_index):
    paginator.find_elements(By.CSS_SELECTOR, 'li')[page_index].click()
    sleep(3)
    load_items()
    sleep(2)

def locate_current_candidato(current_page):
    current_name = get_current_name().lower().replace(' ', '')
    if get_current_name() == '': -1, -1
    open_page(current_page)
    names = sels('ul.artdeco-list li.hiring-applicants__list-item div > div.flex-grow-1.artdeco-entity-lockup__content.ember-view > div:nth-child(1)')
    names = [name.text.split('\n')[0].strip().lower().replace(' ', '') for name in names]
    try:
        name_index = names.index(current_name)
        if name_index  == 0:
            if get_normal_index(current_page, len(names)) == 0:
                return invert(0, len(page_els)), invert(name_index, len(names))
            return current_page - 1, 0
        current_index = invert(name_index, len(names))
        return current_page, current_index - 1
    except ValueError:
        # Se entrar aqui, ele não está nessa página, ou ele não achou, então ou ele retorna que não encontrou, ou retorna a busca da próxima página
        if get_normal_index(current_page, len(page_els)) == 0: 
            open_page(-1)
            return -1, -1
        return locate_current_candidato(current_page-1)

def mover_da_pasta_downloads(curriculo_path):
    list_of_files = glob.glob(r'C:\\Users\xgabr\Downloads\\*')
    latest_file = max(list_of_files, key=os.path.getctime)
    os.makedirs(os.path.dirname(DOWNLOAD_FOLDER), exist_ok=True)
    # curriculo_path = '.'.join(curriculo_path.split('.')[:-1] + latest_file.split('.')[-1])
    curriculo_path += latest_file[latest_file.rfind('.'):]
    os.replace(latest_file, curriculo_path)

def download_info(el_candidato, index_candidato, tries=5):
    global nome_atual
    clicar(el_candidato)
    sleep(2)
    clicar(sel('body'))
    sleep(2)
    nome = sel('h1.display-flex.align-items-center.t-24').text.replace('Candidatura de ', '').split('\n')[0]
    change_current_name(nome)
    nome_atual = nome
    classificacao = sel('button.artdeco-dropdown__trigger.artdeco-dropdown__trigger--placement-bottom.ember-view.artdeco-button--3.t-black').text.split('\n')[0]
    # if classificacao != 'Classificar como':
    #     if classificacao != 'Candidatura inadequada':
    try:
        sel('.msg-overlay-list-bubble-search')
        botao_min = sel('header.msg-overlay-bubble-header .msg-overlay-bubble-header__controls.display-flex button:nth-child(3)')
        clicar(botao_min)
    except NoSuchElementException:
        ...
    if classificacao not in ['Candidatura adequada', 'Talvez']: return False
    more_button = sel('button.artdeco-button--secondary.artdeco-button--muted.artdeco-button--3')
    clicar(more_button)
    sleep(1)
    infos = sels('ul[aria-live="polite"] li')
    email = infos[1].text.split('\n')[0]
    telefone = infos[2].text.split('\n')[0]
    a = sel('.hiring-screening-questions__content-wrapper').text.lower()
    if 'conversação' in a:
        ingles = 'intermediário'
    elif 'proficiente' in a:
        ingles = 'avançado'
    elif 'bilíngue' in a:
        ingles = 'fluente'
    elif 'nenhum' in a:
        ingles = 'nenhum'
    else:
        ingles = 'desconhecido'


    email = re.findall(r'[\w.+-]+@[\w-]+\.[\w.-]+', email)[0]
    link = sel('.hiring-profile-highlights__see-full-profile a').get_attribute('href')
    telefone = re.findall(r'\+\d.+? ', telefone)[0].strip().replace('+55', '')
    sleep(1)
    # ActionChains(driver
    #             ).move_to_element(sel('main .m1')
    #             ).click(
    #             ).send_keys(Keys.HOME
    #             ).move_to_element(sel('main .m1 .artdeco-card:nth-child(2)')
    #             ).move_to_element(sel('main .m1 .artdeco-card:nth-child(3)')
    #             ).perform()
    try:
        baixar_button = sel('a.inline-flex.align-items-center.link-without-visited-state.t-16')
    except:
        try:
            baixar_button = sel('.ui-attachment.ui-attachment--doc')
        except:
            sel('.hiring-resume-viewer__virus-scan-section')
            if tries == 0:
                raise LookupError('Não foi encontrado o currículo para baixar depois de 5 tentativas!')
            # tenta clicar novamente
            sleep(2)
            try:
                clicar(sels('ul.artdeco-list li.hiring-applicants__list-item')[index_candidato+1])
            except IndexError:
                sleep(2)
                clicar(sels('ul.artdeco-list li.hiring-applicants__list-item')[index_candidato+1])
            return download_info(el_candidato, index_candidato, tries-1)
        
    clicar(baixar_button)
    curriculo_path = f'{DOWNLOAD_FOLDER}/{str(get_current_json_index() + 1).zfill(3)}_{nome.replace(" ", "_")}'
    mover_da_pasta_downloads(curriculo_path)
    update_json({'nome_completo': nome,
             'email': email,
             'telefone': telefone,
             'curriculo': curriculo_path,
             'inglês': ingles,
             'link': link
            })
    return True

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [52]:
start()
put_on_classify_by_date()

In [54]:
def processo_ingestao():
    current_page, current_candidato = locate_current_candidato(-1)
    for i in range(current_page, -len(page_els)-1, -1):
        open_page(i)
        parar = False
        els_candidato = sels('ul.artdeco-list li.hiring-applicants__list-item')
        for j in range(current_candidato, -len(els_candidato)-1, -1):
            sleep(1)
            baixado = download_info(els_candidato[j], j)
            if not baixado:
                parar = True
        current_candidato = -1
        if parar:
            _dict = get_json_dict()
            nome_atual = _dict[str(get_current_json_index(_dict))]['nome_completo']
            print('Parei em ' + nome_atual + ' porque não estão mais como válidos.')
            break
        if i == -len(page_els):
            print('Todos nomes baixados com sucesso!')
def rodar_ingestao():
    try:
        processo_ingestao()
    except:
        rodar_ingestao()
# rodar_ingestao()
processo_ingestao()

Todos nomes baixados com sucesso!


In [ ]:
if current.text == 'FULANO':
    paginator.find_elements(By.CSS_SELECTOR, 'li')[-1].click()
    sleep(3)